# 1. Convolutional Neural Networks Visualization in Pytorch

In this kernel, we'll look into a convolutional network, to try and understand how they work by generating images that maximize the activation of the filters in the convolutional layers.
To generate these images, we apply gradient ascent to the inputs.

## Packages and utils functions

Some util functions to load and visualize images

In [ ]:
import numpy as np
import torch
from torch.autograd import Variable
from torch.optim import SGD
from torchvision import models, transforms
import PIL
import os
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

import scipy.ndimage as ndimage

%matplotlib inline

import scipy.ndimage as nd
import PIL.Image
from IPython.display import clear_output, Image, display
from io import BytesIO


def showarray(a, fmt='jpeg'):
    a = np.uint8(np.clip(a, 0, 255))
    f = BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))
    
def showtensor(a):
    mean = np.array([0.485, 0.456, 0.406]).reshape([1, 1, 3])
    std = np.array([0.229, 0.224, 0.225]).reshape([1, 1, 3])
    inp = a[0, :, :, :]
    inp = inp.transpose(1, 2, 0)
    inp = std * inp + mean
    inp *= 255
    showarray(inp)
    clear_output(wait=True)

def plot_images(im, titles=None):
    plt.figure(figsize=(30, 20))
    
    for i in range(len(im)):
        plt.subplot(10 / 5 + 1, 5, i + 1)
        plt.axis('off')
        if titles is not None:
            plt.title(titles[i])
        plt.imshow(im[i])
        
    plt.pause(0.001)
    
normalise = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

normalise_resize = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def init_image(size=(400, 400, 3)):
    img = PIL.Image.fromarray(np.uint8(np.full(size, 150)))
    img = PIL.Image.fromarray(np.uint8(np.random.uniform(150, 180, size)))
    img_tensor = normalise(img).unsqueeze(0)
    img_np = img_tensor.numpy()
    return img, img_tensor, img_np

def load_image(path, resize=False, size=None):
    img = PIL.Image.open(path)
    
    if size is not None:
        img.thumbnail(size, PIL.Image.ANTIALIAS)
        
    if resize:
        img_tensor = normalise_resize(img).unsqueeze(0)
    else:
        img_tensor = normalise(img).unsqueeze(0)
    img_np = img_tensor.numpy()
    return img, img_tensor, img_np

def tensor_to_img(t):
    a = t.numpy()
    mean = np.array([0.485, 0.456, 0.406]).reshape([1, 1, 3])
    std = np.array([0.229, 0.224, 0.225]).reshape([1, 1, 3])
    inp = a[0, :, :, :]
    inp = inp.transpose(1, 2, 0)
    inp = std * inp + mean
    inp *= 255
    inp = np.uint8(np.clip(inp, 0, 255))
    return PIL.Image.fromarray(inp)

def image_to_variable(image, requires_grad=False, cuda=False):
    if cuda:
        image = Variable(image.cuda(), requires_grad=requires_grad)
    else:
        image = Variable(image, requires_grad=requires_grad)
    return image

## Model Creation

Here we load a pretrained VGG-16 model

## Filter Visualization

This function produces an image that maximizes the activation of the filter at *filter_index* in the layer *layer_index*. 

Usually when we train a Neural Net(NN), we use the following steps.
```
while not converge:
    take some inputs
    calucate predictions
    compute loss
    Do gradient descent on loss to update the weights in each layer
```
This procedure is shown in the following picture.
![](./pictures/nn_train.png)
After training a model, one interesting thing we want to do is that we would like to know what excatly each neuron in different layers look like. One way to do that is we start with inputting a picture with random pixels to the NN, and calculate the activations for the neuron that you want to visualize. Note that larger activations mean that the picture contains features that are closer to what is represented by this neuron, thus we want to maximize the activation. So after calculating the activation, we will do gradient ascent on the activation to update the input image.(All the gradients here are taken with respect to inputs, i.e. each pixel's value).

These are the steps described above.
```
intialize a random picture A
while not converge:
    Input A to the NN
    calucate activation for the specific neuron that you want to visualize
    Do gradient ascent on activation to update A
```
This procedure is shown in the following picture.
![](./pictures/filter_visualization.png)



In our exercise, we will use a pretrained model in pytorch called [VGG](https://arxiv.org/abs/1409.1556). 

## Octaver function

The octaver function is used to produce better output images. This procedure is taken from the [Deep Dream code](https://github.com/google/deepdream/blob/master/dream.ipynb). 
We will use it not only for the deep dream algorithm, but also for visualizing the filters.
The gradient ascent algorithm is run on multiple downscaled versions of the image, and the results are upscaled and merged together to get the final image.

![](https://raw.githubusercontent.com/Hvass-Labs/TensorFlow-Tutorials/master/images/14_deepdream_recursive_flowchart.png)
*Figure 1: Flowchart of the octaver function + deep dream, taken from https://github.com/Hvass-Labs/TensorFlow-Tutorials*

In [ ]:
model = models.vgg16()
model.load_state_dict(torch.load("./input/vgg16/vgg16.pth"))

In [ ]:
use_gpu = False
if torch.cuda.is_available():
    use_gpu = True

print(model)

for param in model.parameters():
    param.requires_grad = False

### Octaver Function

VGG model was trained on images of fairly low resolution, 224 * 224 pixels. So when we use images with much larger resolution, the algorithm described will create many small patterns instead of big ones. 

One solution is to downscale the input image to the same size. But such a low resolution is pixelated and ugly. 

Another solution is to repeatedly downscale the original image and run the algorithm on each of the smaller versions of the image. This creates larger patterns in the 
image that are then refined at the higher resolution. 

This flowchart shows roughly the idea. The algorithm is implemented recursively and supports any number of downscaling levels. 
![](./pictures/octaver_fn.png)

The function ```octaver_fn()``` implements what is described above.

In [ ]:
# Finish this function
def octaver_fn(model, base_img, step_fn, octave_n=6, octave_scale=1.4, iter_n=10, **step_args):
    octaves = [base_img]
    # model: pretrained model
    # base_image: the image that you want to modify
    # step_fn: function for doing a single gradient ascent step
    # octave_n: how many times you want to zoom your picture
    # octave_scale: how much you want to zoom your picture each time
    # iter_n: num of gradient ascent steps you want to take 
    # step_args: a dictionary containing all the kwargs for step_fn
    for i in range(octave_n - 1):
        octaves.append(nd.zoom(octaves[-1], (1, 1, 1.0 / octave_scale, 1.0 / octave_scale), order=1))

    detail = np.zeros_like(octaves[-1])
    for octave, octave_base in enumerate(octaves[::-1]):
        h, w = octave_base.shape[-2:]
        
        if octave > 0:
            h1, w1 = detail.shape[-2:]
            detail = nd.zoom(detail, (1, 1, 1.0 * h / h1, 1.0 * w / w1), order=1)
        
        src = octave_base + detail
        
        for i in range(iter_n):
            src = ...

        detail = src.numpy() - octave_base

    return src

The following function ```filter_step()``` implements one single gradient ascent step.

In [ ]:
def filter_step(model, img, layer_index, filter_index, step_size=0.05, display=True, use_L2=False):
    
    mean = np.array([0.485, 0.456, 0.406]).reshape([3, 1, 1])
    std = np.array([0.229, 0.224, 0.225]).reshape([3, 1, 1])
    
    model.zero_grad()
    
    img_var = image_to_variable(torch.Tensor(img), requires_grad=True, cuda=use_gpu)
    optimizer = SGD([img_var], lr=step_size, weight_decay=1e-4)
    
    x = img_var
    for index, layer in enumerate(model.features):
        ...

    output = x[0, filter_index]
    loss = output.norm()
    loss.backward()
    
    if use_L2:
        #L2 normalization on gradients
        mean_square = torch.Tensor([torch.mean(img_var.grad.data ** 2) + 1e-5])
        img_var.grad.data /= torch.sqrt(mean_square)
        img_var.data.add_(img_var.grad.data * step_size)
    else:
        optimizer.step()
    
    result = img_var.data.cpu().numpy()
    result[0, :, :, :] = np.clip(result[0, :, :, :], -mean / std, (1 - mean) / std)
    
    if display:
        showtensor(result)
    
    return torch.Tensor(result)

The function ```visualize_filter()``` visualize one single filter using ```octaver_fun()```.

In [ ]:
def visualize_filter(model, base_img, layer_index, filter_index, 
                     octave_n=6, octave_scale=1.4, iter_n=10, 
                     step_size=0.05, display=True, use_L2=False):
    
    return ...

The function ```show_layer()``` is a helper function to visualize a number of filters for a given layer.

In [ ]:
def show_layer(layer_num, filter_start=10, filter_end=20, step_size=0.05, use_L2=False):
    filters = []
    titles = []
    
    _, _, img_np = init_image(size=(600, 600, 3))
    for i in range(filter_start, filter_end):
        title = "Layer {} Filter {}".format(layer_num , i)
        print(title)
        filter = ...
        filter_img = tensor_to_img(filter)
#         filter_img.save(title + ".jpg")
        filters.append(tensor_to_img(filter))
        titles.append(title)
        
    
    plot_images(filters, titles)
    return filters, titles

Now you have finished all the functions you need to visualize some filters. Your task here is to use the functions above to visualize some filters in some layers.

In the following cell, visualize filter 11 to 15 in layer 1.

In [ ]:
images, titles = ...

In the following cell, visualize filter 11 to 15 in layer 10 (This might take some time to run)

In [ ]:
images, titles = ...

### Question: Compare your visualizations for layer 1 and layer 10, what's the difference in the patterns shown?

## Deep Dream

The Deep Dream function is similar to the filter visualization, but instead of starting from a random noise image, we start from an actual picture and try to maximize the network output. In this way, we're enhancing the features that the network recognizes in the image. 

The function ```make_step``` implements one single gradient ascent step for deap dream algorithm.

In [ ]:
def objective(dst, guide_features):
    if guide_features is None:
        return dst.data
    else:
        x = dst.data[0].cpu().numpy()
        y = guide_features.data[0].cpu().numpy()
        ch, w, h = x.shape
        x = x.reshape(ch, -1)
        y = y.reshape(ch, -1)
        A = x.T.dot(y)
        diff = y[:, A.argmax(1)]
        diff = torch.Tensor(np.array([diff.reshape(ch, w, h)]))
        return diff

def make_step(model, img, objective=objective, control=None, step_size=1.5, end=28, jitter=32):
    mean = np.array([0.485, 0.456, 0.406]).reshape([3, 1, 1])
    std = np.array([0.229, 0.224, 0.225]).reshape([3, 1, 1])
    
    ox, oy = np.random.randint(-jitter, jitter+1, 2)
    
    img = np.roll(np.roll(img, ox, -1), oy, -2)
    tensor = torch.Tensor(img) 
    
    img_var = image_to_variable(tensor, requires_grad=True, cuda=use_gpu)
    model.zero_grad()
      
    x = img_var
    for index, layer in enumerate(model.features.children()):
        ...
    
    delta = objective(x, control)
    x.backward(delta)
    
    #L2 Regularization on gradients
    mean_square = torch.Tensor([torch.mean(img_var.grad.data ** 2)])
    img_var.grad.data /= torch.sqrt(mean_square)
    img_var.data.add_(img_var.grad.data * step_size)
    
    result = img_var.data.cpu().numpy()
    result = np.roll(np.roll(result, -ox, -1), -oy, -2)
    result[0, :, :, :] = np.clip(result[0, :, :, :], -mean / std, (1 - mean) / std)
    showtensor(result)
    
    return torch.Tensor(result)

The function ```deepdream()``` is a wrapper function using the implemented ```octaver_fn()``` on the above step function.

In [ ]:
def deepdream(model, base_img, octave_n=6, octave_scale=1.4, 
              iter_n=10, end=28, control=None, objective=objective, 
              step_size=1.5, jitter=32):
    
    return ...

Let's select a pciture and show it.

In [ ]:
input_img, input_tensor, input_np = load_image('./input/data/market1.jpg', size=[1024, 1024])
print(input_img.size)
input_img

Then we apply the deepdream function on layer 14 on it to see the results.

In [ ]:
dream = ...
dream = tensor_to_img(dream)
dream

## Controlling the dream

We can control the dream by trying to alter the image in order to maximize the filters that are activated by another image (which we'll call guide). The idea would be we input the guide image into the NN and stops at the layer that you want to do deepdream on. The values you got are called guide features. Then when you input your picture which you want to do deepdream on, you would like to make the result on your chosen layer as close to the guide features as possible. (Actually this process has been implemented already, you just need to find out how to use it.)

In [ ]:
guide_img, guide_img_tensor, guide_img_np = load_image('./input/data/kitten2.jpg', resize=True)
plt.imshow(guide_img)

In [ ]:
end = 26

guide_image = image_to_variable(guide_img_tensor, cuda=use_gpu)

# genrate guide features by input the guide image into the NN until the wanted layer.
...
    
dream = deepdream(model, input_np, end=end, step_size=0.06, octave_n=4, control=guide_features)
dream = tensor_to_img(dream)
dream

## Notes

There are other pictures provided in the homework zip, you can try them if you want!

# 2. Feature selection using RF, PCA and Lasso

Here we are reusing the dataset from hw6.

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("./datasets/VideoGameSales.csv")
df.head()

In [ ]:
df = df.drop("Name", axis=1)
df.head()

In [ ]:
from sklearn.model_selection import train_test_split

y = df[["Global_Sales"]].values.ravel()

# Generate dummies for all catagrical features
X = pd.get_dummies(df.drop(["Global_Sales"], axis=1)).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0, shuffle=False)


print("Original:", X.shape)
print("Train:   ", X_train.shape,y_train.shape)
print("Test:    ", X_test.shape,y_test.shape)

## 2.1 PCA

In this section you need to do the following:
 - Fit a PCA with 10 components
 - Draw a plot showing the explained variance ratio for each principle compoenent

In [ ]:
...

In [ ]:
...

### Question: How would you determine the number of principle components to use?

## 2.2 Random Forest

In this section you need to do the following:
 - Fit a Random Forest
 - Draw a barchart showing the feature importance from RF for each feature

In [ ]:
...

In [ ]:
...

### Question: According to feature importance, how would you select your features?

## 2.3 Lasso

In this section you need to do the following:
 - Fit a Lasso with alpha=0.1
 - Draw a barchart showing the weights for each feature

In [ ]:
from sklearn.linear_model import Lasso

...

In [ ]:
...

### Question: According to Lasso, what are the features that need to be deleted? Comparing with RF, do the selected features match?